<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    47
1    14
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1044.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [9]:
stock_prec_dev('^NSEI').tail(1)

[*********************100%***********************]  1 of 1 completed


,Close Price,RSI,Stock,Dev%_200,Slope%_200,Max,Min,Premium%,Discount%
Date,,,,,,,,,
2022-12-05,18701.0,68.0,^NSEI,7.87,0.75,18812.0,15294.0,96.85,3.15


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(20)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-05,669.0,41.0,MOTILALOFS.NS,-13.72,-1.55,980.0,654.0,4.49,95.51,930.0,39.01
35,2022-12-05,412.0,67.0,WIPRO.NS,-9.37,-1.32,722.0,375.0,10.76,89.24,433.0,5.10
4,2022-12-05,3369.0,45.0,DIVISLAB.NS,-11.53,-1.23,4678.0,3266.0,7.28,92.72,4065.0,20.66
15,2022-12-05,80.0,67.0,NATIONALUM.NS,-3.37,-1.20,130.0,67.0,20.95,79.05,91.0,13.75
2,2022-12-05,475.0,44.0,ICICIPRULI.NS,-9.81,-1.14,616.0,447.0,16.75,83.25,647.0,36.21
23,2022-12-05,88.0,64.0,IBREALEST.NS,-3.74,-1.10,179.0,60.0,23.19,76.81,95.0,7.95
7,2022-12-05,554.0,43.0,JUBLFOOD.NS,-6.60,-0.84,786.0,464.0,27.90,72.10,651.0,17.51
36,2022-12-05,1100.0,61.0,TECHM.NS,-5.44,-0.77,1806.0,965.0,16.02,83.98,1156.0,5.09
6,2022-12-05,4056.0,56.0,NAUKRI.NS,-4.31,-0.71,5740.0,3375.0,28.81,71.19,4851.0,19.60
1,2022-12-05,465.0,73.0,GODREJIND.NS,-0.50,-0.64,644.0,405.0,24.96,75.04,638.0,37.20


In [12]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
55,2022-12-05,322.0,67.0,CANBK.NS,32.17,3.69,329.0,175.0,95.78,4.22,310.0,-3.73
21,2022-12-05,137.0,61.0,FEDERALBNK.NS,22.59,1.79,138.0,81.0,98.77,1.23,149.0,8.76
9,2022-12-05,899.0,66.0,AXISBANK.NS,17.39,1.70,915.0,626.0,94.53,5.47,1044.0,16.13
59,2022-12-05,4403.0,77.0,BRITANNIA.NS,18.31,1.59,4433.0,3094.0,97.79,2.21,3935.0,-10.63
13,2022-12-05,617.0,66.0,SBIN.NS,16.69,1.51,617.0,435.0,100.16,-0.16,709.0,14.91
48,2022-12-05,1039.0,62.0,SUNPHARMA.NS,13.97,1.43,1052.0,737.0,96.00,4.00,1056.0,1.64
8,2022-12-05,1196.0,59.0,INDUSINDBK.NS,14.29,1.34,1264.0,778.0,85.91,14.09,1394.0,16.56
11,2022-12-05,934.0,57.0,ICICIBANK.NS,12.89,1.33,953.0,654.0,93.51,6.49,1075.0,15.10
40,2022-12-05,337.0,41.0,ITC.NS,12.25,1.33,361.0,208.0,84.15,15.85,350.0,3.86
18,2022-12-05,1260.0,47.0,M&M.NS,11.49,1.24,1354.0,691.0,85.79,14.21,1404.0,11.43


In [13]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-05,669.0,41.0,MOTILALOFS.NS,-13.72,-1.55,980.0,654.0,4.49,95.51,930.0,39.01
4,2022-12-05,3369.0,45.0,DIVISLAB.NS,-11.53,-1.23,4678.0,3266.0,7.28,92.72,4065.0,20.66
2,2022-12-05,475.0,44.0,ICICIPRULI.NS,-9.81,-1.14,616.0,447.0,16.75,83.25,647.0,36.21
36,2022-12-05,1100.0,61.0,TECHM.NS,-5.44,-0.77,1806.0,965.0,16.02,83.98,1156.0,5.09
6,2022-12-05,4056.0,56.0,NAUKRI.NS,-4.31,-0.71,5740.0,3375.0,28.81,71.19,4851.0,19.60
1,2022-12-05,465.0,73.0,GODREJIND.NS,-0.50,-0.64,644.0,405.0,24.96,75.04,638.0,37.20
33,2022-12-05,1142.0,67.0,MUTHOOTFIN.NS,-0.08,-0.52,1540.0,956.0,31.87,68.13,1207.0,5.69
43,2022-12-05,224.0,50.0,TATAPOWER.NS,-1.23,-0.44,290.0,195.0,30.74,69.26,232.0,3.57
50,2022-12-05,3426.0,65.0,TCS.NS,3.13,0.21,4019.0,2982.0,42.81,57.19,3385.0,-1.20
14,2022-12-05,1919.0,64.0,INDIGO.NS,3.84,0.32,2265.0,1549.0,51.68,48.32,2186.0,13.91


In [14]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     17.353203
1     16.548043
2     16.107651
3     10.106762
4      9.190391
6      8.718861
10     7.099644
14     6.187722
26     3.389680
33     2.531139
36     2.264235
43     1.588078
50    -0.533808
51    -0.551601
Name: Upside%, dtype: float64

In [17]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
35,2022-12-05,412.0,67.0,WIPRO.NS,-9.37,-1.32,722.0,375.0,10.76,89.24,433.0,5.10
7,2022-12-05,554.0,43.0,JUBLFOOD.NS,-6.60,-0.84,786.0,464.0,27.90,72.10,651.0,17.51
23,2022-12-05,88.0,64.0,IBREALEST.NS,-3.74,-1.10,179.0,60.0,23.19,76.81,95.0,7.95
15,2022-12-05,80.0,67.0,NATIONALUM.NS,-3.37,-1.20,130.0,67.0,20.95,79.05,91.0,13.75
37,2022-12-05,622.0,57.0,BERGEPAINT.NS,-3.23,-0.31,791.0,561.0,26.61,73.39,651.0,4.66
42,2022-12-05,4028.0,46.0,DMART.NS,-3.17,-0.48,4821.0,3231.0,50.14,49.86,4177.0,3.70
5,2022-12-05,6660.0,34.0,BAJFINANCE.NS,-2.91,-0.20,7853.0,5284.0,53.58,46.42,7978.0,19.79
27,2022-12-05,1234.0,49.0,HAVELLS.NS,-1.96,-0.16,1418.0,1070.0,47.23,52.77,1327.0,7.54
46,2022-11-22,3434.0,47.0,MINDTREE.NS,-1.89,0.17,4842.0,2742.0,32.95,67.05,3520.0,2.50
12,2022-12-05,429.0,52.0,TATAMOTORS.NS,-1.66,-0.23,525.0,372.0,37.09,62.91,493.0,14.92


In [16]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
4,2022-12-05,3369.0,45.0,DIVISLAB.NS,-11.53,-1.23,4678.0,3266.0,7.28,92.72,4065.0,20.66
35,2022-12-05,412.0,67.0,WIPRO.NS,-9.37,-1.32,722.0,375.0,10.76,89.24,433.0,5.10
7,2022-12-05,554.0,43.0,JUBLFOOD.NS,-6.60,-0.84,786.0,464.0,27.90,72.10,651.0,17.51
36,2022-12-05,1100.0,61.0,TECHM.NS,-5.44,-0.77,1806.0,965.0,16.02,83.98,1156.0,5.09
37,2022-12-05,622.0,57.0,BERGEPAINT.NS,-3.23,-0.31,791.0,561.0,26.61,73.39,651.0,4.66
27,2022-12-05,1234.0,49.0,HAVELLS.NS,-1.96,-0.16,1418.0,1070.0,47.23,52.77,1327.0,7.54
46,2022-11-22,3434.0,47.0,MINDTREE.NS,-1.89,0.17,4842.0,2742.0,32.95,67.05,3520.0,2.50
57,2022-11-29,4912.0,54.0,LTI.NS,-1.56,-0.22,7562.0,3824.0,29.11,70.89,4615.0,-6.05
29,2022-12-05,3157.0,53.0,ASIANPAINT.NS,0.16,-0.03,3576.0,2580.0,57.92,42.08,3361.0,6.46
50,2022-12-05,3426.0,65.0,TCS.NS,3.13,0.21,4019.0,2982.0,42.81,57.19,3385.0,-1.20


In [23]:
# turning around 
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_200'] < 1)].sort_values(by = 'Slope%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
22,2022-12-05,882.0,66.0,GODREJCP.NS,4.07,0.12,968.0,665.0,71.78,28.22,958.0,8.62
60,2022-12-05,729.0,49.0,IRCTC.NS,1.40,0.14,903.0,571.0,47.62,52.38,571.0,-21.67
46,2022-11-22,3434.0,47.0,MINDTREE.NS,-1.89,0.17,4842.0,2742.0,32.95,67.05,3520.0,2.50
50,2022-12-05,3426.0,65.0,TCS.NS,3.13,0.21,4019.0,2982.0,42.81,57.19,3385.0,-1.20
34,2022-12-05,2255.0,61.0,DEEPAKNTR.NS,6.69,0.24,2660.0,1720.0,56.96,43.04,2371.0,5.14
25,2022-12-05,4479.0,55.0,DRREDDY.NS,3.26,0.25,4907.0,3719.0,64.01,35.99,4829.0,7.81
20,2022-12-05,800.0,57.0,TATACONSUM.NS,3.69,0.26,849.0,660.0,74.34,25.66,871.0,8.88
14,2022-12-05,1919.0,64.0,INDIGO.NS,3.84,0.32,2265.0,1549.0,51.68,48.32,2186.0,13.91
49,2022-12-05,1598.0,51.0,COLPAL.NS,2.52,0.32,1678.0,1394.0,71.92,28.08,1606.0,0.50
10,2022-12-05,589.0,70.0,HDFCLIFE.NS,3.68,0.35,694.0,504.0,44.68,55.32,683.0,15.96
